<a href="https://colab.research.google.com/github/GISChat/Fine-tune-bert/blob/main/GISchat_bert_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 训练

Thanks to


*   https://github.com/yemanzhongting/NoiseExposure/blob/main/fine_tune_bert.py
*   https://huggingface.co/datasets/dirtycomputer/weibo_senti_100k
* https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/weibo_senti_100k/intro.ipynb
* bert base chinese : https://huggingface.co/google-bert/bert-base-chinese








在微调 BERT 模型时，通常直接使用 BertTokenizer 对文本进行分词和编码，因为它与 BERT 模型的预训练词汇表和编码格式完全兼容。

In [ ]:
# @title 安装必要的包
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 req

In [ ]:
# @title 必要的权限验证（请填写有write权限的token）
from huggingface_hub import notebook_login

notebook_login()

## 主要过程

In [ ]:
# @title 加载微博100k数据集
from datasets import load_dataset
import pandas as pd

ds = load_dataset("dirtycomputer/weibo_senti_100k")
ds

Generating train split:   0%|          | 0/119988 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 119988
    })
})

In [ ]:
# @title
# 查看数据集数据集有一个默认的分割，'train'
df = pd.DataFrame(ds['train'])
df

,label,review
0,1,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你]
1,1,@张晓鹏jonathan 土耳其的事要认真对待[哈哈]，否则直接开除。@丁丁看世界 很是细心...
2,1,姑娘都羡慕你呢…还有招财猫高兴……//@爱在蔓延-JC:[哈哈]小学徒一枚，等着明天见您呢/...
3,1,美~~~~~[爱你]
4,1,梦想有多大，舞台就有多大![鼓掌]
...,...,...
119983,0,一公里不到，县医院那个天桥下右拐200米就到了！//@谢礼恒: 我靠。这个太霸道了！离224...
119984,0,今天真冷啊，难道又要穿棉袄了[晕]？今年的春天真的是百变莫测啊[抓狂]
119985,0,最近几天就没停止过！！！[伤心]
119986,0,//@毒药女流氓:[怒] 很惨!


In [ ]:
# @title 划分训练集和测试集
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

# 划分训练集和测试集
train_df, test_df = train_test_split(df, test_size=0.2,shuffle=True, random_state=42)
print('train_df info')
print('test_df info')
print(train_df)
print(test_df)

train_df info
test_df info
        label                                             review
43852       1               乐乐的武汉之武汉步行街~[哈哈]我窃以为只比南京的夫子庙好了那么一丢丢~
21608       1        →_→//@最爱Cruise: 然后去医院医生给火柴包起一圈绷带出来就成棉签了[哈哈]
91972       0                                   午饭来不及吃，先垫下肚子。[泪]
78263       0  [汗]//@LimingX:[衰] //@万昊-百度: @cremer_xiao 想想办法吧...
73639       0                      昨天被人恶作剧了，表白什么的不是真的? [晕][晕][晕]
...       ...                                                ...
110268      0  谢谢！亲爱的 @逗豆芽儿 收到了你从三亚送来的礼物~ 喜欢咖啡的我自然地爱吃咖啡糖，其他三亚...
119879      0                     [衰]除了最后一张，几大主持都被黑了一遍~小编你是要闹哪样？
103694      0                                解散，彻底解散！！！[衰][衰][衰]
860         1  回复@薛晓棠:哎，我咋记得这提壶是牛奶的意思？不会真的让我提个壶逮谁浇谁吧，那叫浇朋友，我懂...
15795       1                         这么早回家，真是不可思议，这可是周六呀！！！[嘻嘻]

[95990 rows x 2 columns]
       label                                             review
47063      1  回复@沂蒙山区老母鸡:如何解决保鲜问题？ //@沂蒙山区老母鸡:我想办法给您快递好吗？[太开...
34050      1         幻想的吧，大中午的。 //@小树林s

In [ ]:
# @title 检查设备可用性 初始化分词器
# 检查是否有可用的 CUDA 设备
if torch.cuda.is_available():
    # 指定使用 cuda:1
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# 初始化分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [ ]:
# @title 创建数据集
class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
# 创建类别到索引的映射
label_to_index = {label: idx for idx, label in enumerate(df['label'].unique())}
# 使用映射来更新标签
train_labels = [label_to_index[label] for label in train_df['label']]
test_labels = [label_to_index[label] for label in test_df['label']]
print(f'train_labels:{train_labels}')
print(f'test_labels:{test_labels}')
# 然后在创建数据集时使用这些更新后的标签
train_dataset = TextDataset(tokenizer, train_df['review'].tolist(), train_labels)
test_dataset = TextDataset(tokenizer, test_df['review'].tolist(), test_labels)


# 创建数据集 如果所有的标签都是数字 本部分也可以直接采用下面的两行
# train_dataset = TextDataset(tokenizer, train_df['text'].tolist(), train_df['label'].tolist())
# test_dataset = TextDataset(tokenizer, test_df['text'].tolist(), test_df['label'].tolist())


print(train_dataset)
print(test_dataset)

train_labels:[0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,

In [ ]:
# @title 设定参数
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=len(df['label'].unique()))
model = model.to(device)# 将模型转移到对应的设备运行

# 定义训练参数
training_args = TrainingArguments(
    output_dir='GISchat-weibo-100k-fine-tuned-bert',
    num_train_epochs=1,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1000,
    evaluation_strategy="steps",
    eval_steps=500,  # 每500步进行一次评估
    save_strategy = "steps",
    save_steps= 1000,
    # evaluation_strategy ="epoch",
    # save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id = "wsqstar/GISchat-weibo-100k-fine-tuned-bert"
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# @title 训练
trainer.train()

trainer.evaluate()


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,No log,0.074877,0.980582,0.980559,0.969157,0.992232
1000,0.105300,0.039729,0.983915,0.983966,0.968438,1.000000
1500,0.105300,0.039707,0.983957,0.984007,0.968517,1.000000
2000,0.048600,0.044348,0.983957,0.984007,0.968517,1.000000
2500,0.048600,0.039072,0.984040,0.984089,0.968676,1.000000
3000,0.037600,0.036185,0.984040,0.984089,0.968676,1.000000


{'eval_loss': 0.03618502616882324,
 'eval_accuracy': 0.9840403366947246,
 'eval_f1': 0.9840887374849404,
 'eval_precision': 0.9686758812464219,
 'eval_recall': 1.0,
 'eval_runtime': 277.1632,
 'eval_samples_per_second': 86.584,
 'eval_steps_per_second': 1.353,
 'epoch': 1.0}

In [ ]:
# @title 保存模型到本地
# 保存模型到本地/或者，下面是远程的方式：这里需要提供自己的用户名和自己的repository(这里不需要自己手动新建，有自动建立的)
# output_dir = "wsqstar/GISChat-weibo-100k-fine-tuned-bert"
# # model.save_pretrained(output_dir)
# # tokenizer.save_pretrained(output_dir)
# pr

In [ ]:
# @title 上传
# trainer.push_to_hub()
repo_id = "GISChat-weibo-100k-fine-tuned-bert"
tokenizer.push_to_hub(repo_id)

README.md:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wsqstar/GISchat-weibo-100k-fine-tuned-bert/commit/0d4b73a877e89a15feaa012cc635799805537ad7', commit_message='Upload tokenizer', commit_description='', oid='0d4b73a877e89a15feaa012cc635799805537ad7', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# !rm -rf /content/GISchat-weibo-100k-fine-tuned-bert/*

# 测试
# https://huggingface.co/wsqstar/GISchat-weibo-100k-fine-tuned-bert

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip show tqdm

Name: tqdm
Version: 4.66.5
Summary: Fast, Extensible Progress Meter
Home-page: https://tqdm.github.io
Author: 
Author-email: 
License: MPL-2.0 AND MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: cmdstanpy, dopamine_rl, gdown, google-generativeai, huggingface-hub, hyperopt, ipyparallel, kaggle, kagglehub, moviepy, nltk, panel, proglog, prophet, spacy, tensorflow-datasets, torchtext, transformers


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# 加载预训练的BERT模型和分词器
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 假设有2个类别

# 准备要分类的文本
texts = ["姑娘都羡慕你呢…还有招财猫高兴…", "好冷啊", "好害怕"]

# 对文本进行tokenize并转换为模型输入格式
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# 使用模型进行推理
with torch.no_grad():
    outputs = model(**inputs)

# 获取预测结果
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)

# 输出预测类别
for i, text in enumerate(texts):
    print(f"文本: {text}")
    print(f"预测类别: {predictions[i].item()}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


文本: 姑娘都羡慕你呢…还有招财猫高兴…
预测类别: 0
文本: 好冷啊
预测类别: 0
文本: 好害怕
预测类别: 0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("wsqstar/GISchat-weibo-100k-fine-tuned-bert")
model = AutoModelForSequenceClassification.from_pretrained("wsqstar/GISchat-weibo-100k-fine-tuned-bert")

# 准备要分类的文本
texts = ["非常喜欢你", "你好令人讨厌","好害怕"]

# 对文本进行tokenize并转换为模型输入格式
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# 使用模型进行推理
with torch.no_grad():
    outputs = model(**inputs)
    print(outputs)

# 获取预测结果
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)

# 输出预测类别//'
for i, text in enumerate(texts):
    print(f"文本: {text}")
    print(f"预测类别: {predictions[i].item()}")


SequenceClassifierOutput(loss=None, logits=tensor([[ 3.1651, -3.3074],
        [ 2.8844, -3.0833],
        [ 2.9839, -3.0992]]), hidden_states=None, attentions=None)
文本: 非常喜欢你
预测类别: 0
文本: 你好令人讨厌
预测类别: 0
文本: 好害怕
预测类别: 0


In [ ]:
import torch.nn.functional as F

# 应用softmax函数
probabilities = F.softmax(logits, dim=-1)

# 获取概率最高的类别索引
predictions = torch.argmax(probabilities, dim=-1)

# 输出预测类别及其概率
for i, text in enumerate(texts):
    print(f"文本: {text}")
    predicted_index = predictions[i].item()
    print(f"预测类别索引: {predicted_index}")
    print(f"预测类别概率: {probabilities[i, predicted_index].item()}")

文本: 非常喜欢你
预测类别索引: 0
预测类别概率: 0.9984570741653442
文本: 你好令人讨厌
预测类别索引: 0
预测类别概率: 0.9974466562271118
文本: 好害怕
预测类别索引: 0
预测类别概率: 0.9977241158485413
